In [10]:
from pylab import *
import matplotlib.pyplot as plt
import netCDF4
import numpy as np 

In [11]:
def read(his_file,sim_num,fname):
    print his_file
    print "Read veg files"
    nc = netCDF4.Dataset(his_file)
    mask = nc.variables['mask_rho'][:]

# read x,y
    x = nc.variables['x_rho'][:,:]
    y = nc.variables['y_rho'][:,:] 
# Remember all arrays are (time,kmax,jmax,imax) contrary to Matlab     
    imax=size(x[1,:])
    jmax=size(y[:,1])
# read water depth
    depth = nc.variables['h'][:,:]
    theta_s = nc.variables['theta_s'][:]
    theta_b = nc.variables['theta_b'][:]
    depth_c = nc.variables['hc'][:]
    
#Get Stretching function 
    N=10 # Number of vertical levels, kgrid=1 MATLAB code copy 
    Np=N+1 
    ds=1.0/N
    s=np.zeros(11)
    Csur=np.zeros(11)
    for i in range(0,Np):
        lev=i
        s[i]=(lev-N)*ds
        Csur[i]=-s[i]**2
    
# Reshape 1D vertical variables to 3D so we can broadcast
    s.shape=(np.size(s),1,1)
    Csur.shape=(np.size(Csur),1,1)
    
    tidx = -1       # Fetching data from last time step 

# read a 2D water level (height of ocean surface) at specified time step
    eta = nc.variables['zeta'][tidx,:,:] 
        
# calculate the 3D field of z values (vertical coordinate) at this time step
    z0=(depth_c*s+depth*Csur)/(depth_c+depth) 
    z = eta+(eta+depth)*z0
    
# detect the presence of vegetation patch to know its extents (Currently hardwired
    imin_plant=41 ; imax_plant=50 ; jmin_plant=45 ; jmax_plant=54
    
# Calculate wave dissipation due to vegetation just behind the patch 
# calculate it along a cross section at integrate along depth 
# Take 5 points on both sides of the edges 
    vol_dissip_veg=0.0 # Initialize volume integrated dissipation due to vegetation 
    dissip_veg =nc.variables['Dissip_veg'][tidx,:,:]
 
    for j in range(jmin_plant-5,jmax_plant+6):
        for i in range(imin_plant-5,imax_plant+6):
            dy=y[j+1,i]-y[j,i]
            dx=x[j,i+1]-x[j,i]
            for k in range(1,Np): 
                dz = z[k,j,i]-z[k-1,j,i]          # change in water level at last time step 
                vol_dissip_veg = dissip_veg[j,i]*dx*dy*dz+vol_dissip_veg#*dx*dy*dz + vol_dissip_veg 
                
# Calculate water level change at the surface at the end of 2 days of simulation 
# Water level change in the middle transect before and after the patch
# This is at the last time step
    imid=45
    water_lev_change=z[N,jmin_plant,imid]-z[N,jmax_plant,imid]
    #print water_lev_change
    
# Calculate change in kinetic energy before and after the patch
    u=nc.variables['u'][tidx,:,:,:]
    v=nc.variables['v'][tidx,:,:,:]
    w=nc.variables['w'][tidx,:,:,:]
    
    bef_j=jmin_plant-5
    aft_j=jmax_plant+5
    
    tke_bef=0.0 
    tke_aft=0.0 
    
    for k in range(1,N):
        for i in range(imin_plant-5,imax_plant+6):
            u_c=0.5*(u[k,bef_j,i]   + u[k,bef_j,i+1])
            v_c=0.5*(v[k,bef_j+1,i] + v[k,bef_j,i])
            w_c=0.5*(w[k,bef_j,i]   + w[k-1,bef_j,i])
            tke_bef=sqrt(u_c*u_c+v_c*v_c+w_c*w_c)+tke_bef
            
            u_c=0.5*(u[k,aft_j,i]   + u[k,aft_j,i+1])
            v_c=0.5*(v[k,aft_j+1,i] + v[k,aft_j,i])
            w_c=0.5*(w[k,aft_j,i]   + w[k-1,aft_j,i])
            tke_aft=sqrt(u_c*u_c+v_c*v_c+w_c*w_c)+tke_aft
        
    delta_tke=tke_aft-tke_bef
    
    f.write("%s\t %s\t, %s\t, %s\t \n" % (sim_num, vol_dissip_veg, water_lev_change, delta_tke))


In [12]:
'''Output from a simulations based one effective quadratures inputs in 
vegetation module COAWST 
# Total wave dissipation due to vegetation in (Watt/m2 * m3) = Watt-mts. u
nits of volume integrated vegetation dissipation around the vegetation patch

# Water level change at the free surface at a middle transect along lateral axis
going through the vegetation patch (units m)

# Total kinetic energy change in a plane before and after the vegetation 
''' 
fname='veg_output.txt'
f = open(fname, 'w')
total_sim_list=16
f.write("Case_no, Dissipation(W-mt), Water level change(mt),  delta(TKE): \n")
#f.write("Random numbers are: \n")        
for sim_num in range(1,total_sim_list):
    path1='/media/gadar/DATADRIVE1/sensitivity_history/'
    path2='veg_test_his_'+ str(sim_num)+'.nc'
    url=path1+path2
    read(url,sim_num,fname) 
    
f.close()

/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_1.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_2.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_3.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_4.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_5.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_6.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_7.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_8.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_9.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_10.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_11.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_12.nc
Read veg files
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_13.nc
Read veg f

In [13]:
 
a1=1
b1=2
with open('Output.txt', 'w') as f:
        f.write("Random numbers are: \n")
        f.write("%s,%s\n" % (a1,b1))